In [ ]:
### Run cells one at a time

In [1]:
import os
import json
from selenium import webdriver
from wdf import FormFiller

os.environ["SELENIUM_MANAGER_DISABLE_TELEMETRY"] = "true"

from dotenv import load_dotenv
load_dotenv()

driver = webdriver.Chrome()
driver.get("data:,")

f = FormFiller(driver)

In [ ]:
### Paste in URL, click "Apply manually"

In [ ]:
email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")

f.create_acct_or_log_in(email, password)

In [ ]:
### Make sure it logged you in

In [3]:
f.fill_out_information_page()

In [ ]:
### Review info entered, click "Next"

In [4]:
with open("resume.json") as rf:
    experience_data = json.load(rf)

f.fill_out_experience_page(experience_data)

In [ ]:
### That's all folks

### Work in progress:

In [ ]:
import os
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException

from bs4 import BeautifulSoup

In [ ]:
null_container = f.wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[aria-labelledby='Education-section'] div[data-fkit-id$='--null']")))[-1]
labels = null_container.find_elements(By.CSS_SELECTOR, "label")

[label.get_attribute('for') for label in labels]

# for nyp job:
# ['education-238--schoolName',
#  'education-238--degree',
#  'education-238--fieldOfStudy',
#  None,
#  None]

# for jj job:
# ['education-162--school',
#  'education-162--degree',
#  'education-162--fieldOfStudy']

['education-162--school',
 'education-162--degree',
 'education-162--fieldOfStudy']

In [ ]:
el = null_container.find_element(By.ID, 'education-162--school')

element_html = driver.execute_script("return arguments[0].outerHTML;", el)
pretty_html = BeautifulSoup(element_html, "html.parser").prettify()
pretty_html.replace(" ", "\n")